In [5]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import re

In [2]:
df = pd.read_csv('Assets\CONVICTIONSUMMARY_270_opt3_best2(1).csv', sep='|')

In [16]:
df2 = pd.DataFrame()
df2 = df.loc[df['TYPE'] == 'MERGED']

Extract Time and Date

In [ ]:
df2["Time"] = df2.apply(lambda row: re.split(" ",row.DATE)[0], axis=1)

df2["DATE"] = df2.apply(lambda row: re.split(" ",row.DATE)[-1],axis=1)

In [18]:
df2.dtypes

DATE             object
ID               object
STOCK            object
SECTOR           object
TYPE             object
CONVICTIONS     float64
Time             object
dtype: object

In [19]:
df2['DATE'] = pd.to_datetime(
                          df2['DATE'],
                          format='%Y-%m-%d')

<ipython-input-19-3660468bee21>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['DATE'] = pd.to_datetime(


In [41]:
df2 = df2.sort_values(by="DATE")

## Adding returns

In [22]:
list_of_companies = df2["STOCK"].unique()

In [23]:
print(list_of_companies)

['SU' 'GGG' 'WGR' ... 'GBX' 'OVTI' 'SNCAF']


In [55]:
start_date = df2.DATE.min()
end_date = df2.DATE.max()

In [53]:
start_date = start_date + pd.DateOffset(months=6)
print(start_date)

2004-08-11 00:00:00


In [59]:
import yfinance as yf

#define the ticker symbol
tickerSymbol = 'SU'

#get data on this ticker
tickerData = yf.Ticker(tickerSymbol)

#get the historical prices for this ticker
tickerDf = tickerData.history(period='1m', start=start_date, end=end_date)

#see your data
tickerDf["STOCK"] = tickerSymbol
tickerDf

,Open,High,Low,Close,Volume,Dividends,Stock Splits,STOCK
Date,,,,,,,,
2004-02-10,8.120848,8.370671,8.108198,8.332724,1641400,0.0,0,SU
2004-02-11,8.411782,8.452892,8.304263,8.402295,1280800,0.0,0,SU
2004-02-12,8.383321,8.383321,8.187257,8.196744,950400,0.0,0,SU
2004-02-13,8.301100,8.405457,8.073413,8.114523,1313400,0.0,0,SU
2004-02-17,8.285291,8.291615,8.149311,8.212557,1207000,0.0,0,SU
...,...,...,...,...,...,...,...,...
2004-07-07,7.921155,8.247898,7.898949,8.181281,2736200,0.0,0,SU
2004-07-08,8.216178,8.225695,8.108321,8.209833,1169400,0.0,0,SU
2004-07-09,8.216176,8.396995,8.184453,8.381134,1180000,0.0,0,SU


In [61]:
new_df = pd.merge(df2, tickerDf,  how='left', left_on=['STOCK','DATE'], right_on = ['STOCK','Date'])